In [1]:
# + tags=["parameters"]
# declare a list tasks whose products you want to use as inputs
upstream = ['vectorizer_countVec']
random_seed = 42

In [2]:
# Parameters
random_seed = 42
upstream = {
    "vectorizer_countVec": {
        "nb": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/vectorizer_countVec.ipynb",
        "vectorizer": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/vectorizer_countVec.pkl",
    }
}
product = {
    "nb": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/topic_modeling_disaster_type_final.ipynb",
    "lda_model_earthquake": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/lda_model_earthquake.pkl",
    "lda_model_fire": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/lda_model_fire.pkl",
    "lda_model_flood": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/lda_model_flood.pkl",
    "lda_model_hurricane": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/lda_model_hurricane.pkl",
    "lda_topics_disaster_type": "/Users/mboussarov/_umsi/Capstone/umads_697_data_medics/pipeline/output/lda_topics_disaster_type.csv",
}


importing all the functions defined in tools_rjg.py

In [3]:
from tools_rjg import *

In [4]:
import pandas as pd
import re, nltk, spacy, gensim
nltk.download('punkt')
import pickle

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1129)>


In [5]:
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD

In [6]:
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
df_all = generate_disaster_type_dataframe(disaster_types = ('earthquake', 'fire', 'flood', 'hurricane')
                                          , dev_train_test= ('dev', 'train', 'test'))

In [8]:
df_all['tweet_text_cleaned'] = df_all['tweet_text'].apply(lambda x: tweet_preprocessing(x))
df_all['lemmatized'] = df_all['tweet_text_cleaned'].apply(lambda x: lemmatize_tweet_text(x, allowed_postags=('NOUN', 'ADJ', 'VERB', 'ADV')))

In [9]:
df_all.sample(100)

,tweet_id,tweet_text,class_label,disaster_type,tweet_text_cleaned,lemmatized
15938,909194537171636224,"Carnival, Arisons Pledge $10 Million to Hurric...",rescue_volunteering_or_donation_effort,hurricane,"[carnival, arisons, pledge, million, hurricane...",carnival arison pledge relief
1866,912107426329001986,RT @keithrhernandez: I have set up a Hurricane...,other_relevant_information,hurricane,"[have, hurricane, maria, fundraiser, page, hon...",hurricane fundraiser page honor father match d...
1658,1112148115610460160,@DonnellySports thoughts on the Hoiberg hire? ...,not_humanitarian,flood,"[thoughts, hoiberg, hire, know, hear, some, ne...",thought hire hear input
1821,903056992243257346,"Flooding in India, Bangladesh, Nepal. Massive ...",caution_and_advice,hurricane,"[flooding, india, bangladesh, nepal, massive, ...",flood nepal massive rainfall harvey icecap mel...
5562,901721245435252736,Deaths reported as Harvey strikes Texas coast:...,injured_or_dead_people,hurricane,"[deaths, reported, harvey, strikes, texas, coa...",death report harvey strike late
...,...,...,...,...,...,...
27860,1168247170090319877,"@pulte With that said if I’m chosen, I pledge ...",rescue_volunteering_or_donation_effort,hurricane,"[with, that, said, chosen, pledge, donate, hur...",say choose pledge donate hurricane relief almo...
300,729655286554832896,RT @mayordavepelham: Thanks Fed Gov’t for matc...,rescue_volunteering_or_donation_effort,fire,"[thanks, matching, donations, cross, wildfire,...",thank match donation cross wildfire relief effort
1849,1022857305694588929,Greece is more than grateful for Koreas help d...,sympathy_and_support,fire,"[greece, more, than, grateful, koreas, help, d...",greece more grateful korea help difficult time...
7655,1167952719199424512,Golf: Hurricane Dorian sends golf courses brac...,infrastructure_and_utility_damage,hurricane,"[golf, hurricane, dorian, sends, golf, courses...",golf hurricane send golf course brace potentia...


## LDA Model

In [10]:
params_earthquake = {'n_components' : 2,
          'max_iter' : 10,
          'learning_method' : 'online',
          'random_state' : random_seed,
          'batch_size' : 128,
          'evaluate_every' : -1,
          'n_jobs' : 1,
          'learning_decay' : .5}

params_fire = {'n_components' : 4,
          'max_iter' : 10,
          'learning_method' : 'online',
          'random_state' : random_seed,
          'batch_size' : 128,
          'evaluate_every' : -1,
          'n_jobs' : 1,
          'learning_decay' : .5}

params_flood = {'n_components' : 3,
          'max_iter' : 10,
          'learning_method' : 'online',
          'random_state' : random_seed,
          'batch_size' : 128,
          'evaluate_every' : -1,
          'n_jobs' : 1,
          'learning_decay' : .5}

params_hurricane = {'n_components' : 5,
          'max_iter' : 10,
          'learning_method' : 'online',
          'random_state' : random_seed,
          'batch_size' : 128,
          'evaluate_every' : -1,
          'n_jobs' : 1,
          'learning_decay' : .5}

lda_model_earthquake = LatentDirichletAllocation(**params_earthquake)
lda_model_fire = LatentDirichletAllocation(**params_fire)
lda_model_flood = LatentDirichletAllocation(**params_flood)
lda_model_hurricane = LatentDirichletAllocation(**params_hurricane)


In [11]:
with open("output/vectorizer_countVec.pkl", "rb") as f:
    vectorizer = pickle.load(f)

/Users/mboussarov/Library/Caches/pypoetry/virtualenvs/data-medics-capstone-ZGa4TAVF-py3.9/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator CountVectorizer from version 1.1.1 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [12]:
earthquake_text_vectorized = vectorizer.transform(list(df_all[df_all['disaster_type']=='earthquake']['lemmatized']))
fire_text_vectorized = vectorizer.transform(list(df_all[df_all['disaster_type']=='fire']['lemmatized']))
flood_text_vectorized = vectorizer.transform(list(df_all[df_all['disaster_type']=='flood']['lemmatized']))
hurricane_text_vectorized = vectorizer.transform(list(df_all[df_all['disaster_type']=='hurricane']['lemmatized']))

/Users/mboussarov/Library/Caches/pypoetry/virtualenvs/data-medics-capstone-ZGa4TAVF-py3.9/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['donate'] not in stop_words.
  warnings.warn(


In [13]:
lda_model_earthquake.fit_transform(earthquake_text_vectorized)

array([[0.77503453, 0.22496547],
       [0.89939596, 0.10060404],
       [0.04631994, 0.95368006],
       ...,
       [0.81555754, 0.18444246],
       [0.16263193, 0.83736807],
       [0.91068813, 0.08931187]])

In [14]:
lda_model_fire.fit_transform(fire_text_vectorized)

array([[0.12500127, 0.62499527, 0.12500152, 0.12500194],
       [0.02272852, 0.02290931, 0.93150542, 0.02285676],
       [0.03603028, 0.60705595, 0.03580154, 0.32111223],
       ...,
       [0.40663193, 0.51443582, 0.06672117, 0.01221109],
       [0.92447282, 0.02536235, 0.02500018, 0.02516466],
       [0.05077452, 0.09224879, 0.58222756, 0.27474913]])

In [15]:
lda_model_flood.fit_transform(flood_text_vectorized)

array([[0.17840612, 0.76961967, 0.05197422],
       [0.05453876, 0.77655022, 0.16891101],
       [0.37543094, 0.54121059, 0.08335847],
       ...,
       [0.70572085, 0.05447348, 0.23980567],
       [0.08338713, 0.58432106, 0.33229182],
       [0.52783743, 0.19424844, 0.27791413]])

In [16]:
lda_model_hurricane.fit_transform(hurricane_text_vectorized)

array([[0.01551876, 0.19008961, 0.68668435, 0.0923226 , 0.01538469],
       [0.01544671, 0.01538469, 0.01538468, 0.70762439, 0.24615953],
       [0.16918277, 0.09226605, 0.0153847 , 0.70763337, 0.01553311],
       ...,
       [0.86531364, 0.03356784, 0.03404396, 0.03333341, 0.03374115],
       [0.59999481, 0.25407556, 0.01854691, 0.10905586, 0.01832686],
       [0.02005398, 0.24511754, 0.26831516, 0.24647487, 0.22003845]])

In [17]:
with open(product['lda_model_earthquake'], "wb") as f:
    pickle.dump(lda_model_earthquake, f)
with open(product['lda_model_fire'], "wb") as f:
    pickle.dump(lda_model_fire, f)
with open(product['lda_model_flood'], "wb") as f:
    pickle.dump(lda_model_flood, f)
with open(product['lda_model_hurricane'], "wb") as f:
    pickle.dump(lda_model_hurricane, f)

## Performance Stats

In [18]:
# Log Likelyhood: Higher the better
print("Log Likelihood Earthquake: ", lda_model_earthquake.score(earthquake_text_vectorized))
print("Log Likelihood Fire: ", lda_model_fire.score(fire_text_vectorized))
print("Log Likelihood Flood: ", lda_model_flood.score(flood_text_vectorized))
print("Log Likelihood Hurricane: ", lda_model_hurricane.score(hurricane_text_vectorized))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity Earthquake: ", lda_model_earthquake.perplexity(earthquake_text_vectorized))
print("Perplexity Fire: ", lda_model_fire.perplexity(fire_text_vectorized))
print("Perplexity Flood: ", lda_model_flood.perplexity(flood_text_vectorized))
print("Perplexity Hurricane: ", lda_model_hurricane.perplexity(hurricane_text_vectorized))

Log Likelihood Earthquake:  -488653.45987242664


Log Likelihood Fire:  -862991.27309673


Log Likelihood Flood:  -1017564.9037416612


Log Likelihood Hurricane:  -3262678.1450290033


Perplexity Earthquake:  1515.1718974502674


Perplexity Fire:  1864.339285829549


Perplexity Flood:  1798.312954109303


Perplexity Hurricane:  2185.718504207312


## Extract Topic Keywords

In [19]:
topics_earthquake = pd.DataFrame(show_topics(fitted_vectorizer=vectorizer, fitted_lda_model=lda_model_earthquake, n_words=100, dname='earthquake'))
topics_fire = pd.DataFrame(show_topics(fitted_vectorizer=vectorizer, fitted_lda_model=lda_model_fire, n_words=100, dname='fire'))
topics_flood = pd.DataFrame(show_topics(fitted_vectorizer=vectorizer, fitted_lda_model=lda_model_flood, n_words=100, dname='flood'))
topics_hurricane = pd.DataFrame(show_topics(fitted_vectorizer=vectorizer, fitted_lda_model=lda_model_hurricane, n_words=100, dname='hurricane'))

df_topic_keywords = pd.concat([topics_earthquake,topics_fire,topics_flood,topics_hurricane],axis=0)
    # df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
    # df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords = df_topic_keywords.transpose()
df_topic_keywords.head()

/Users/mboussarov/Library/Caches/pypoetry/virtualenvs/data-medics-capstone-ZGa4TAVF-py3.9/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/mboussarov/Library/Caches/pypoetry/virtualenvs/data-medics-capstone-ZGa4TAVF-py3.9/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/mboussarov/Library/Caches/pypoetry/virtualenvs/data-medics-capstone-ZGa4TAVF-py3.9/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. P

,earthquake_Topic 0,earthquake_Topic 1,fire_Topic 0,fire_Topic 1,fire_Topic 2,fire_Topic 3,flood_Topic 0,flood_Topic 1,flood_Topic 2,hurricane_Topic 0,hurricane_Topic 1,hurricane_Topic 2,hurricane_Topic 3,hurricane_Topic 4
Word 0,pray,help,death,relief,home,help,help,relief,flooding,storm,evacuation,damage,emergency,help
Word 1,prayer,donate,toll,support,lose,camp,need,donate,state,damage,say,pray,safe,relief
Word 2,safe,death,death toll,help,destroy,victim,relief,victim,damage,destroy,life,home,supply,need
Word 3,injure,victim,camp,make,burn,donate,rescue,fund,disaster,wind,destruction,cause,rescue,donate
Word 4,thought,rescue,dead,donation,family,blame,food,help,year,flooding,disaster,time,stay,victim


In [20]:
df_topic_keywords.to_csv(product['lda_topics_disaster_type'])